# Laboratorio 2, Alejo Eyzaguirre


# Problema 1: Llaves

Se crea un algoritmo eficiente capaz de rápidamente resolver el problema. Para ello se utilizan dos listas, `espadas_sobrantes` y `bestias_no_matadas`, cada una de estas corresponde a una lista de 26 elementos cada una. El elemento 1 de la primera lista, corresponde a la espada **a** y el elemento 26 a la espada **z**, lo mismo con la lista de `bestias_no_matadas`pero corresponde a las letras mayúsculas.

La función número de espadas entonces, recorre el string "mapa", cada dos pasos. El elemento **i** corresponde a la espada y el elemento **i+1** corresponde a la bestia a enfrentar. Se revisa entonces si NO se puede asesinar a la bestia del tipo **i+1** con la espada **i** si es este el caso entonces la lista `espadas_sobrantes`, ve que en la posición, correspondiente al número de la letra de la espada en el abecedario, aumenta en 1. 

Luego se pregunta, si se tiene guardada alguna espada (recogida anteriormente) para poder matar a la bestia que tenemos al frente, si es este el caso, entonces se utiliza la espada y disminuye en 1 el número de espadas de ese tipo en la lista `espadas_sobrantes`.

Si la espada no es igual a la bestia que se tiene al frente, y tampoco tenemos una espada (recogida anteriormente) para poder matar esta misma bestia, entonces aumenta en 1 el número de bestias no matadas del tipo de la bestia que tenemos al frente.

Después de haber recorrido todo el string, evaluando cada espada y bestia en cada etapa, quedamos con que la suma de todos los elementos de la lista `bestias_no_matadas`, corresponde al número de espadas mínimas que se debe comprar antes de partir la *travesía*. Este es el número que retorna la función.

In [1]:
import time
import string

mapa = 'kSuXaInEiEoRxTkVhBzMdNpUiJmVlIvEzVhKoAiPtUkKdZoMgSdJeKwEePdZoSeTrRkOtKjTpVxPzHwWsIlXcToShHrBtNjNcNwK'

def numero_espadas(mapa):
    #Cada elemento en las listas corresponde a el número de espadas no usadas o bestias no matadas
    #del tipo "letra".EXPLICAR EN CUADRO DE TEXTO.
    
    espadas_sobrantes = [0 for _ in range(26)]
    bestias_no_matadas = [0 for _ in range(26)]
    
    for i in range(0,len(mapa),2):
 
        espada = mapa[i]
        bestia = mapa[i+1]
        
        if espada.upper() != bestia:

            espadas_sobrantes[ord(espada)-97] += 1           
            
            if espadas_sobrantes[ord(bestia)-65] > 0:
                espadas_sobrantes[ord(bestia)-65] = espadas_sobrantes[ord(bestia)-65] - 1
            
            else:
                bestias_no_matadas[ord(bestia)-65] += 1

    
    return sum(bestias_no_matadas)

inicio = time.time()       
min_espadas = numero_espadas(mapa)
print(min_espadas)
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')  
    

25
la funcion se demoro un total de 0.0005292892456054688 segundos


# Problema 2.A: El Jinete de las Montañas (A*)

En este desarrollo se utilizará la implementación de A*, sugerida en la materia de clases (https://github.com/jrialland/python-astar). Para ello se instala inicialmente la librería astar.

Teniendo en cuenta el README.rst se define la clase `Astar_Caballo` que hereda de la clase padre `AStar`. Luego se definen distintos atributos propios de la clases "hija", como es el horse_map (la matriz de alturas de cada ubicación geográfica), que tiene un sierto ancho y alto. Se procede con definir los distintos métodos (@abstract_methods) de la clase Astar_Caballo que son las siguientes:

1) `neighbours`: Este método recibe un nodo del horse_map y entrega sus vecinos correspondientes. Para ello hay que tener sumo cuidado con no quedar "out of range" y estar seguro que el código no falla al probar distintos nodos de origen y destino más en los bordes del mapa.

2) `distance_between`: Este método recibe dos nodos y calcula el costo entre ellos, para luego retornarlo. En este ejercicio en particular, el costo venía dado por la diferencia de las alturas más uno, si la altura del segundo nodo (nodo destino inmediato) es mayor a la del primero (nodo actual), sino la función retorna 1. 

3) `heuristic_cost_estimate`: Este método recibe dos nodos y calcula el **costo heurístico** entre ellos, para luego retornarlo. Se consideró que una medida adecuada para este costo, sería la distancia euclidiana dividida por √2, ya que nunca será mayor al costo para alcanzar el objetivo (recordar que en este caso si se permite avanzar en línea diagonal). Para darme cuenta de eso, utilicé un enfoque iterativo y al ver que la distancia euclidiana fallaba en encontrar la ruta mínima, atiné a dividir esta en √2 y se logró un resultado coherente, y con la distancia de un paso en diagonal normalizado a 1. De esta forma se mantiene una función, que no es muy costosa de computar, computacionalmente hablando y que entrega una solución que permite que A* pueda converger al mínimo.

Se define por último el método `is_goal_reached`, que retorna True si el nodo actual es igual al nodo de origen. Habiendo implementado todos estos métodos en la clase hija `Astar_Caballo`, tenemos finalmente un algoritmo de A* que puede resolver el problema.


In [ ]:
!pip3 install astar==0.92

In [ ]:
from astar import AStar
import math 
import time


class Astar_Caballo(AStar):
    """
    La clase Astar_Caballo hereda de la clase AStar. Astar_Caballo recibe como input solamente el mapa.
    
    """
    def __init__(self, mapa_caballo):
        super().__init__()       
        self.mapa_caballo = mapa_caballo
        self.ancho = len(mapa_caballo[0])
        self.alto = len(mapa_caballo)
        
    def neighbors(self, node):
        """
        node es tupla (f, c) que representa un elemento de la matriz. 
        Evalúo quienes son los vecinos del nodo, teniendo mucho ojo en no entregar "vecinos out of range" 
        especialmente en los bordes de la matriz.
        """
        f_0, c_0 = node
        validos = []
        for delta_f in range(-1,2):
            for delta_c in range(-1,2):
                f_1 = f_0 + delta_f
                c_1 = c_0 + delta_c                
                if f_1 >= 0 and f_1 < self.alto and c_1 >= 0 and c_1 < self.ancho and (f_1, c_1) != node:
                    validos.append((f_1, c_1))
        return validos
 
    def distance_between(self, n1, n2):
        """
        Calculo la distancia entre cada nodo, en este ejercicio en particular la "distancia" estaba
        determinada por la diferencia de alturas entre cada nodo, o mejor dicho cada punto de la matriz.
        """
        f1, c1 = n1
        f2, c2 = n2
        if self.mapa_caballo[f2][c2] > self.mapa_caballo[f1][c1]:
            costo = self.mapa_caballo[f2][c2]- self.mapa_caballo[f1][c1] + 1
        else:
            costo = 1
        return costo      
                                   
    def heuristic_cost_estimate(self, n1, n2):
        """
        La heurística (que diferencia A* de Djiskstra) que se utiliza es la Distancia Euclidiana entre 
        dos tuplas (n1 y n2). Se normaliza por raíz de dos para poder asegurarnos que la heurística entre
        dos nodos es siempre menor o igual a el costo de trasladarse entre dos nodos.
        """
        (f1, c1) = n1
        (f2, c2) = n2
        #Retorno la distancia euclidiana entre ambos nodos, y normalizo las diagonales dividiendo por raíz de 2.
        return (math.hypot(f2 - f1, c2 - c1)/math.sqrt(2))
           
    def is_goal_reached(self, current, goal):
        """
        Esta función evalúa si el nodo actual es el nodo de destino (goal), si es así entonces A* 
        finaliza su trabajo.
        """        
        if current == goal:
            return True 
        

def ruta_de_jaime(origen, destino, mapa):    
    """
    Encuentro primero el camino que recorre nuestro caballo según A*. En cada nodo inspecciona el costo
    agregado (heurística más costo energía caballo) de moverse hacia cada vecino. Avanza hacia el nodo
    con el menor costo.
    Luego se calcula la energía consumida por el caballo al recorrer este camino.
    """    
    camino = list(Astar_Caballo(mapa).astar(origen,destino))  
    energia = 0
    for i in range(len(camino)-1):
        f0, c0 = camino[i]
        f1, c1 = camino[i+1]
        if mapa[f1][c1] > mapa[f0][c0]:
            energia += mapa[f1][c1] - mapa[f0][c0] + 1
        else:
            energia += 1      
    return (camino, energia)

origen = (1,2)
destino = (5,6)
mapa = [[31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37]]
    
inicio = time.time()  
ruta_minima = ruta_de_jaime(origen, destino, mapa)
print(ruta_minima)
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')
                                   

In [ ]:
origen = (1,2)
destino = (99,75)

#Creación de Mapa grande para Evaluar el tiempo de ejecución con un mapa gigante.
mapa = [[31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],     
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],     
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],     
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],     
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],     
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37],     
        [31, 27, 32, 24, 26, 34, 32, 24, 31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24,31, 27, 32, 24, 26, 34, 32, 24],
        [44, 3, 2, 46, 45, 6, 25, 38, 44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38,44, 3, 2, 46, 45, 6, 25, 38],
        [14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31, 14, 9, 36, 39, 46, 15, 5, 31],
        [44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1, 44, 18, 32, 34, 9, 3, 2, 1],
        [23, 10, 38, 46, 2, 19, 5, 39, 23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39,23, 10, 38, 46, 2, 19, 5, 39],
        [7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25, 7, 44, 13, 40, 20, 1, 19, 25],
        [32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50, 32, 9, 25, 41, 23, 14, 50, 50],
        [18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37, 18, 35, 45, 17, 34, 35, 6, 37]           
       ]


print(len(mapa[0]))
print(len(mapa))

mapa_1 = [[26, 15, 5, 43, 45, 5, 36, 9, 24, 23, 27, 12, 9, 10, 21, 33, 1], [49, 10, 43, 26, 44, 26, 7, 5, 41, 23, 32, 21, 5, 5, 13, 17, 1], [15, 34, 8, 28, 39, 45, 27, 29, 21, 29, 44, 8, 23, 19, 41, 19, 3], [21, 9, 31, 16, 17, 37, 42, 35, 42, 18, 39, 11, 1, 10, 27, 14, 22], [24, 21, 22, 50, 19, 6, 17, 20, 32, 33, 39, 40, 21, 43, 10, 5, 42], [29, 45, 34, 30, 41, 25, 45, 25, 50, 6, 47, 43, 5, 43, 48, 17, 11], [27, 43, 9, 14, 3, 16, 47, 12, 5, 21, 30, 26, 23, 42, 12, 20, 36], [38, 38, 13, 6, 13, 28, 1, 49, 19, 12, 48, 33, 42, 8, 46, 19, 10], [46, 17, 32, 48, 23, 13, 43, 24, 23, 29, 21, 25, 10, 44, 42, 46, 42], [48, 18, 33, 7, 5, 6, 29, 32, 13, 6, 20, 49, 30, 38, 49, 2, 9], [34, 5, 46, 40, 31, 37, 16, 46, 26, 36, 15, 26, 1, 50, 8, 37, 22], [27, 39, 38, 42, 22, 23, 50, 36, 31, 28, 47, 21, 1, 37, 42, 24, 23], [26, 35, 36, 42, 15, 43, 25, 9, 16, 28, 25, 24, 9, 43, 37, 43, 16], [18, 36, 3, 16, 20, 49, 19, 15, 24, 23, 50, 46, 39, 8, 50, 16, 36], [12, 45, 40, 20, 44, 43, 42, 22, 11, 43, 32, 3, 21, 10, 31, 30, 25], [47, 44, 43, 12, 35, 19, 16, 13, 26, 7, 36, 42, 7, 24, 27, 40, 22], [1, 4, 31, 41, 25, 12, 22, 41, 17, 12, 16, 16, 42, 12, 28, 37, 20]]
[(3, 2), (4, 2), (5, 3), (6, 3), (7, 4), (8, 5), (9, 6), (9, 7), (10, 8), (11, 9), (12, 9), (13, 9), (14, 10), (15, 11)]

print(len(mapa_1[0]))
print(len(mapa_1))




# Pregunta 2B: Löbel Finder (Árbol):

En la siguiente pregunta se aplican los conceptos aprendidos de la materia de árbol. Intenté buscar una buena implementación de Trie pero no pillé, así que armé uno *casero*. Este consiste en la creación de una clase `nodo` que contiene como atributos: una letra, un padre, una frecuencia, una freq_pal y un diccionario de hijos (en que la llave es una letra y los valores los nodos hijos correspondientes a esa letra). El atributo frecuencia denota la frecuencia de la búsqueda con mayor frecuencia que hay al tomar ese camino del árbol (para saber después que camino tomar al aplicar el Löbel Finder). El atributo, freq_pal en cambio, denota la frecuencia de la palabra que termina en ese nodo, si en ese nodo no termina ninguna palabra la freq_pal es cero.

Se crea una forma de representar estos nodos, que permite visualizar el árbol de una forma ordenada y con un método  DFS (Depth First Search).

Para poder crear el árbol de una manera recursiva, se utiliza la función `recursive_create`, que agrega cada búsqueda al árbol, separando cada caracter y los convierte en nodo agregandoles sus respectivos atributos de frecuencia, freq_pal y de sus respectivos hijos.

In [91]:
class Nodo:

    def __init__(self, padre, letra, frecuencia):
        self.letra = letra
        self.padre = padre
        self.frecuencia = frecuencia
        self.freq_pal = 0
        self.hijos = {}

    def __repr__(self):
        def recorrer_arbol(raiz, profundidad=0):
            for hijo in raiz.hijos.values():
                self.ret += "{}Arista: {} --> freq: {} ---> freq_pal= {} \n".format("\t"*profundidad, hijo.letra, hijo.frecuencia, hijo.freq_pal, self.letra)
                recorrer_arbol(hijo, profundidad+1)

        self.ret = 'HIJOS:\n'
        recorrer_arbol(self)
        return self.ret

In [93]:
def recursive_create(palabra, frecuencia, nodo):
    nodo.frecuencia = max(frecuencia, nodo.frecuencia)
    if len(palabra) == 0:     
        nodo.freq_pal = frecuencia
        return

    letra = palabra[0]    
    if not letra in nodo.hijos:        
        nodo.hijos[letra] = Nodo(nodo, letra, 0)
        
    recursive_create(palabra[1:], frecuencia , nodo.hijos[letra])

In [94]:
raiz = Nodo(None, "", 0)

In [95]:
palabra = "yo"

recursive_create(palabra,4, raiz)

palabra = "yapos cumpa"

recursive_create(palabra, 12, raiz)

In [96]:
palabra = "hola"

recursive_create(palabra, 10 ,raiz)

palabra = "holas"
recursive_create(palabra, 10, raiz)

palabra = "h"
recursive_create(palabra, 25, raiz)

In [97]:
len(raiz.hijos['h'].hijos)

1

In [98]:
raiz.hijos['y'].hijos['a'].hijos['p'].frecuencia

12

In [99]:
print(raiz)

HIJOS:
Arista: y --> freq: 12 ---> freq_pal= 0 
	Arista: o --> freq: 4 ---> freq_pal= 4 
	Arista: a --> freq: 12 ---> freq_pal= 0 
		Arista: p --> freq: 12 ---> freq_pal= 0 
			Arista: o --> freq: 12 ---> freq_pal= 0 
				Arista: s --> freq: 12 ---> freq_pal= 0 
					Arista:   --> freq: 12 ---> freq_pal= 0 
						Arista: c --> freq: 12 ---> freq_pal= 0 
							Arista: u --> freq: 12 ---> freq_pal= 0 
								Arista: m --> freq: 12 ---> freq_pal= 0 
									Arista: p --> freq: 12 ---> freq_pal= 0 
										Arista: a --> freq: 12 ---> freq_pal= 12 
Arista: h --> freq: 25 ---> freq_pal= 25 
	Arista: o --> freq: 10 ---> freq_pal= 0 
		Arista: l --> freq: 10 ---> freq_pal= 0 
			Arista: a --> freq: 10 ---> freq_pal= 10 
				Arista: s --> freq: 10 ---> freq_pal= 10 



### Recorriendo Árbol

Es trivial notar que se utizará una forma de DFS para recorrer el árbol. De esta forma al avanzar con este método en el árbol se podrá decidir mejor que camino tomar según las frecuencias de cada nodo. El método repr ya tiene implementado este método de *Depth First Search*.

Por lo tanto en este recorrido se debe revisar primero que las letras coincidan con las del prefijo entregado, en el mismo orden, y que siempre se apunte a la dirección o al hijo que tiene un nodo con mayor frecuencia (habiendo cumplido ya la primera condición).

Para seleccionar la búsqueda, que parta con el prefijo ingresado, con mayor frecuencia, se utiliza la función `seleccion_pro`. Esta recibe como inputs un nodo y un prefijo y sigue los siguientes pasos. Primero toma el camino en el árbol, correspondiente al de el prefijo. Al haber terminado el prefijo (ya se compararon todas las letras del prefijo con las que tiene seguir en el árbol), se procede a avanzar según el camino con mayor frecuencia. Entonces para cada hijo del nodo, se evaluará cual tiene la mayor frecuencia y se avanza en esa dirección (de manera recursiva). Sin embargo, si el nodo en que se esta parado tiene una freq_pal mayor que el hijo, no se avanza más y se retorna la búsqueda que termina en ese nodo. 

In [101]:
def seleccion_pro(nodo, prefijo):       
    if len(prefijo)!= 0:        
        letra_p = prefijo[0]
        hijo = nodo.hijos[letra_p]        
        ret = letra_p + seleccion_pro(hijo, prefijo[1:])
        return ret
    else:
        max_freq = 0
        max_hijo = None
        for hijo in nodo.hijos.values(): 
            if hijo.frecuencia > max_freq:
                max_freq = hijo.frecuencia  
                max_hijo = hijo
        if max_hijo is not None and max_hijo.frecuencia > nodo.freq_pal:
            ret = max_hijo.letra + seleccion_pro(max_hijo, "")
            return ret
        return ""

In [109]:
seleccion_pro(raiz, "y")

'yapos cumpa'

### Fuerza Bruta (no usada):

La siguiente función (`seleccion`) es una que cree, en base de listas, pero al final no la uso... la dejo igual porque me sirve para entender bien los pasos que sigue mi función seleccion_pro que es casi igual.

In [ ]:
#Seleccion con listas...

respuesta = []
def seleccion(nodo, prefijo):       
    if len(prefijo)!= 0:        
        letra_p = prefijo[0]
        hijo = nodo.hijos[letra_p]
        respuesta.append(letra_p)
        seleccion(hijo, prefijo[1:])
        return
    else:
        max_freq = 0
        max_hijo = None
        for hijo in nodo.hijos.values(): 
            if hijo.frecuencia > max_freq:
                max_freq = hijo.frecuencia  
                max_hijo = hijo
        if max_hijo is not None:
            respuesta.append(max_hijo.letra)
            seleccion(max_hijo, "")
            return
        return

### motor_lobel:

Finalmente se crea la función `motor_lobel`, que fusiona en una única función el procesamiento de los inputs (prefijos e histórico), la creación del árbol (con recursive_create) y la selección del camino con mayor frecuencia correspondiente al prefijo con seleccion_pro.

In [110]:
def motor_lobel(prefijos, historico):
    #Primero creo arbol usando mi función recursive create.
    #creo raíz del árbol...
    raiz = Nodo(None, "", 0)    
    for tupla in historico:
        frecuencia, palabra = tupla
        recursive_create(palabra, frecuencia, raiz)
    #Con el arbol creado selecciono la búsqueda más frecuente que parten con los prefijos entregados.    
    lista = list()
    for prefijo in prefijos:
        try: 
            lista.append(seleccion_pro(raiz, prefijo))
        except:
            lista.append(prefijo)  
    return lista
        
    

### Simulacion:

Se corre la simulación con las respectivas funciones y además se toma el tiempo de ejecución de la función `motor_lobel`.

In [111]:
import time

prefijos = ['ll', 'ñandu', 'es']
historico = [(5, 'llama'),
             (6, 'llamar'),
             (12, 'llueve'),
             (4, 'llueve hoy'),
             (7, 'es'),
             (5, 'estructura'),
             (2, 'estas'),
             (6, 'estas aca')]

inicio = time.time()  
recomendaciones = motor_lobel(prefijos, historico) 
print(recomendaciones)
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')

['llueve', 'ñandu', 'es']
la funcion se demoro un total de 0.0005669593811035156 segundos


In [114]:
#Para usar otros inputs!
from random import sample, randint

def get_rec_info():
    # esta funcion retorna una lista con tuplas de la frecuencia y la frase asociada y
    # tambien una lista de prefijos asociada, para la cual deberian poder dar una 
    # recomendacion en menos de 1.5 segundos (en colab).
    # es importante que el archivo 'info_recomendaciones.txt' este el la misma carpeta que
    # el script. En colab se pueden subir archivos con el navegador en la izquierda. 

    with open('info_recomendaciones.txt', 'r') as file:
        ret = [(int(line[0]), ' '.join(line[2:])) for line in [l.strip().split() for l in file]]
    prefijos = sample(list(set(pal[:randint(1,9*len(pal)//10)] for pal in [p[1] for p in ret])), 3000)
    return ret, prefijos

historico_2, prefijos_2 = get_rec_info()



In [113]:
inicio = time.time()  
recomendaciones = motor_lobel(prefijos_2, historico_2) 
print(recomendaciones)
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')

['gave him a hard look', 'every single day of the', 'of how to use the', 'our cake and eat it', 'was proud that he had', 'the quest for the historical', 'if you want to find', 'is due partly to the', 'over a very short period', 'too far in favor of', 'enough to keep her busy', 'factors may account for the', 'want to wake up in', 'benefit from an increase in', 'spending time with us this', 'how bad will it be', 'been charged with war crimes', 'of which there are many', 'frowned and shook her head', 'permanent resident of the united', 'the problem is not that', 'of men and women who', 'the team reported in the', 'development in the west bank', 'are beginning to see more', 'participate in some of the', 'take a day or two', 'in the wrong direction and', 'rather than engaging in a', 'in every county in the', 'it was also the most', 'who had already made a', 'he was so focused on', 'a critical part of the', 'three standard deviations from the', 'public citizen v united states', 'success or f

# Problema 3: Traductor Extraterrestre

En el siguiente problema se resolverá el problema con un algoritmo de backtracking. Para ello se debe crear una función de "avanzar" de manera recursiva y otra de "retroceder" para deshacer el "paso en falso", cometido hacia una dirección equivocada.

Primero se procederá a definir las distintas funciones necesarias para implementar el backtracking.

### Función "`es_valido`": 

Si bien no defino una función que señale si el estado actual es valido o no, en esta primera parte, defino una función que entrega las posibles movidas válidas para el elemento *x* del input estructura. Esta función se llama `movidas_validas` y entrega a patir de una cierto índice de comienzo correspondiente a una letra de la estructura determinado, los posibles índices de término para el elemento siguiente por asignar de la estructura.Este recibe un resultado parcial, la estructura y el mensaje. 

Resultado parcial, corresponde a los índices de término de cada uno de los elementos de la estructura. Es necesario entonces que resultado parcial, tenga siempre un largo menor o igual a la cantidad de elementos que tiene estructura (número de letras mayúsculas). Entonces si la estructura tiene solo 3 letras mayúsculas, y el resultado parcial es de la siguiente forma [2, 5, 7], esto implica que el primer slice correspondiente a la primera letra mayúscula será mensaje[0:2], el segundo será mensaje[2:5] y el tercero será mensaje[5:7].

Antes que todo eso sí se define la función `estructura_pro` que agarra el string estructura, y lo convierte en una lista que solo considera las letras mayúsculas (omite los espacios).

In [115]:
mensaje = 'elaguaeselaguaeselpez'
estructura = 'A B B C'

def estructura_pro(estructura):
    estructura = estructura.split(" ")
    return estructura

In [116]:
mensaje[0:-4]

'elaguaeselaguaese'

In [117]:
mensaje[0:len(mensaje) - 4 + 1]

'elaguaeselaguaesel'

In [118]:
list(list(range(0,1)) for _ in range(10))

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]

In [119]:
def movidas_validas(resultado_parcial, estructura, mensaje):
    # resultado parcial con forma [0, indice_termino_0, indice_termino_1, ...]
    # estructura es lista ["a", "b", "b", "c"]
    return list(range(1+resultado_parcial[-1], 1+len(mensaje)-len(estructura)+len(resultado_parcial)))
    

En el siguiente ejemplo es posible notar, que para un resultado parcial de [0, 4, 7], la función `movidas_validas` que recibe la estructura y el mensaje del enunciado (*elaguaeselaguaeselpez*), retorna la siguiente lista como output:

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

Esta lista entonces señala que el segundo término "B" de la estructura, podría ir del 7 al 20. No puede terminar en el 21 porque todavía falta por agregar el término "C", que debe representar al menos una letra. Esto se puede notar más claramente al ver el cuadro de cógido subsiguiente.

In [120]:
resultado_parcial = [0, 4, 7]
estructura_arreglada = estructura_pro(estructura)
print(estructura_arreglada)
validas = movidas_validas(resultado_parcial, estructura_arreglada, mensaje)
print(validas)

['A', 'B', 'B', 'C']
[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [121]:
[mensaje[resultado_parcial[-1]:tope] for tope in validas]

['s',
 'se',
 'sel',
 'sela',
 'selag',
 'selagu',
 'selagua',
 'selaguae',
 'selaguaes',
 'selaguaese',
 'selaguaesel',
 'selaguaeselp',
 'selaguaeselpe']

### Función `es_valido` propiamente tal:

Esta función señala si el resultado parcial que recibe es "legal". Para ello debe cumplir ciertas condiciones. La función `es_valido` debe retornar True si el largo de resultado parcial es menor o igual al largo de estructura. Además si resultado parcial tiene largo igual a el largo de la estructura más uno y además el último elemento de resultado parcial es igual a el largo del mensaje, entonces también se debe retornar True. Si no se cumple ninguna de las dos condiciones entonces retorna False.

La función se compone solo de eso, dado que con la definición de la función `movidas_validas`, podemos ya saber que los avances del backtracking serán "legales", y no requieren una doble confirmación.

In [59]:
def es_valido(resultado_parcial, estructura, mensaje):
    #Es valido cuando el resultado parcial tiene un largo igual a estructura.    
    if len(resultado_parcial) <= len(estructura):
        return True
    elif len(resultado_parcial) == (len(estructura) + 1) and resultado_parcial[-1] == len(mensaje): 
        return True
    
    else:
        return False

### Función `es_solucion`:

La función `es_solucion` señala si el resultado parcial recibido puede ser o no una solución. Para ello debe cumplir ciertas condiciones.

En primer lugar es necesario que el largo de resultado parcial sea igual al largo de la estructura (modificada con la función `estructura_pro`) más uno. Además debe ser cierto que el último elemento de la lista sea igual a el largo de el mensaje.

En segundo lugar es necesario que para todo elemento de la estructura igual (misma letra mayúscula), el slice del mensaje que representan deben ser idéntico. Es decir, debe ser cierto que si en la estructura hay dos elementos "A", ambos slice correspondientes deben representar el mismo string. Para este segundo paso defino la clase `Elemento`, que permite hacer que una letra mayúscula sea una instancia del objeto Elemento, y que tiene como atributo los índices en que aparecen en la estructura modificada. 

Además se definen la función `elementizar` que convierte en un Elemento a cada letra de la estructura, y agrega sus respectivos índices correspondientes, agregando estos objetos a un diccionario que guarda toda la información.

In [37]:
class Elemento:
    
    def __init__(self, letra):
        self.letra = letra
        self.indices = []
    
    def agregar_indice(self, string):
        for i in range(len(string)):
            if string[i] == self.letra:
                self.indices.append(i)

In [90]:
def elementizar(estructura):
    dicc = dict()
    for elem in estructura:
        cosa = Elemento(elem)
        cosa.agregar_indice(estructura)
        dicc[elem] = cosa.indices       
    return dicc

elementos = elementizar(estructura)

Las siguientes dos funciones se crean con el objetivo de hacer el código de backtracking más limpio y sencillo. La primera, `parcial_lista` agrega a una lista los slices del string mensaje, que contiene resultado parcial.

La función `formato_pedido` en cambio, a partir de la estructura y la lista que retorna la función `parcial_lista`, entrega el output en el formato pedido (*tupla de tuplas*).

In [39]:
def parcial_lista(resultado_parcial, estructura, mensaje):
    lista = []
    lista.append(mensaje[0:resultado_parcial[1]])
    for i in range(1, len(resultado_parcial)-1):
        lista.append(mensaje[resultado_parcial[i]:resultado_parcial[i+1]])        
    return lista

def formato_pedido(estructura, lista_de_slices):
    final = []
    for j in range(len(estructura)):
        if (estructura[j], lista_de_slices[j]) not in final:
            final.append((estructura[j], lista_de_slices[j]))            
    return (tuple(final))

### Función `es_estado_solución` propiamente tal

Con todo esto armado, podemos crear la función `es_estado_solucion`, y revisar efectivamente si resultado parcial cumple o no con las condiciones para ser una solución del problema propuesto. Para ello se aprovecha las características de un set para ir revisando si los distintos índices de la estructura, de una misma letra, representan el mismo string (slice de mensaje). Por cada letra mayúscula, se agrega cada uno de los slices que representa a un set. Si representan el mismo string, el largo del set debiera ser 1, sino mayor que 1... de esta forma se confirma si la instancia de resultado parcial, es o no solución.

In [84]:
def es_estado_solucion(resultado_parcial, estructura, mensaje):
    
    if len(resultado_parcial) != (len(estructura)+1) or resultado_parcial[-1]!=(len(mensaje) ):
        return False
    
    #Hago una lista que contiene cada slice del string correspondiente a cada letra de la estructura.
    lista = parcial_lista(resultado_parcial, estructura, mensaje)        
    elementos = elementizar(estructura)
    
    #Reviso que los slices del tipo X sean iguales entre sí...
    for elem in elementos.values():
        seteo = set()
        for indiz in elem:
            agrego = lista[indiz]
            seteo.add(agrego)        
        if len(seteo) > 1:
            return False
    
    return True   

In [85]:
def resolver(resultado_parcial, estructura, mensaje):
    
    if not es_valido(resultado_parcial, estructura, mensaje):
        return False
        
    if es_estado_solucion(resultado_parcial, estructura, mensaje):
        return True
    
    else:        
        for movida in movidas_validas(resultado_parcial, estructura, mensaje):         
            resultado_parcial.append(movida)
            nuevo = resultado_parcial               
            
            if resolver(nuevo, estructura, mensaje):                    
                lista = parcial_lista(resultado_parcial, estructura, mensaje)
                ret = formato_pedido(estructura,lista)                
                return ret
    
            else:
                resultado_parcial.pop()
        return False
        

In [122]:
def traduccion(mensaje, estructura):    
    estructura = estructura_pro(estructura)    
    return resolver([0], estructura, mensaje)

traduccion(mensaje, estructura)

(('A', 'e'), ('B', 'laguaese'), ('C', 'lpez'))

In [123]:
estructura2 = "A B A"
mensaje2 = "gatocongato"

inicio = time.time()  
print(traduccion(mensaje2, estructura2))
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')


(('A', 'gato'), ('B', 'con'))
la funcion se demoro un total de 0.001046895980834961 segundos


In [124]:
inicio = time.time()  
print(traduccion(mensaje, estructura))
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')

(('A', 'e'), ('B', 'laguaese'), ('C', 'lpez'))
la funcion se demoro un total de 0.004992008209228516 segundos


In [125]:
#input
mensaje4 = 'vyvlgpldgplddyxywgpld'
estructura4 = 'A G G B G'
#output

inicio = time.time()  
print(traduccion(mensaje4, estructura4))
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')

(('A', 'vyvl'), ('G', 'gpld'), ('B', 'dyxyw'))
la funcion se demoro un total de 0.11412310600280762 segundos


In [88]:
#input
mensaje5 = 'fansahfansahyftslbgvkhkdysdvkhkdxkbot'
estructura5 = 'C C E B D G D F'


inicio = time.time()  
print(traduccion(mensaje5, estructura5))
tiempo = time.time() - inicio
print(f'la funcion se demoro un total de {tiempo} segundos')

(('C', 'fansah'), ('E', 'y'), ('B', 'ft'), ('D', 's'), ('G', 'lbgvkhkdy'), ('F', 'dvkhkdxkbot'))
la funcion se demoro un total de 243.9398422241211 segundos
